### Rules:
- There are 18 weeks
- Each team plays 17 games
- 1 game is against a team in the other conference, the other 16 are in conference
- 1 week is a bye week
- Each team must play the other 3 in their conference 2 times, once at home and once away

In [1]:
import copy, json, random

class NFLSchedule():
    """
    Holds the master schedule for the NFL Season, optional parameter weeks set to default = 16
    """
    def __init__(self, weeks = 18):
        self.schedule = {}

    def __str__(self):
        return json.dumps(self.schedule, indent=4)

    def add_matchup(self, week, hometeam, awayteam):
        self.schedule[week] = {"home": hometeam, "away": awayteam}


class NFLTeam():

    def __init__(self, name, conference, division):
        self.name = name
        self.conference = conference
        self.division = division
        self.bye_week = None
        self.games_played = 0
        self.division_games_played = 0
        self.conference_games_played = 0
        self.schedule = {i: None for i in range(0,19)}
        self.opponents_not_faced = []

    def __repr__(self):
        return f"{self.name} {self.conference}.{self.division[0].lower()}"

    def __str__(self):
        # return f"{self.name} ({self.conference} {self.division}) - Bye Week: {self.bye_week} - Games Played: {self.games_played}"
        return self.__repr__()

    def play_game(self, opponent, week_number, home_indicator):
        self.games_played += 1

        if opponent.conference == self.conference:
            self.conference_games_played += 1
            if opponent.division == self.division:
                self.division_games_played += 1

        # delete the opponent from the list of other teams that this team has not played against
        self.opponents_not_faced.remove(opponent)

        # add this game to the team objects schedule
        self.schedule.append({
            'week': week_number,
            'opponent': opponent,
            'home_game': home_indicator  # You can determine if it's a home game based on your scheduling logic
        })

    def add_game(self, week, opponent):
        self.schedule[int(week)] = opponent
        

    def set_bye_week(self, week_num):
        self.bye_week = week_num
        self.schedule[self.bye_week] = " -- BYE WEEK --"


In [2]:
def add_game_to_team(home_team, away_team):
    """accesses the pre-defined object for both teams and adds this matchup to the team objects scheduule
    Args:
        home_team (class <NFLTeam>): the NFLTeam object that represents the home team for this game
        away_team (class <NFLTeam>): the NFLTeam object that represents the away team for this game
    """
    # check if its a divisional or conference game
    if home_team.conference == home_team.conference:
        home_team.conference_games_played += 1
        away_team.conference_games_played += 1
        # if in conference, check if in division. can only be in division if also in conference
        if home_team.division == home_team.division:
            home_team.division_games_played += 1
            away_team.division_games_played += 1
    # home_team.s

def assign_bye_weeks(league):    
    """Creates the bye weeks and assign each team's .bye_week attribute
        Bye Week Rules:
            - byes occur from week 4 through weeek 14
            - 2 or 4 teams have byes in any given week
            - must be same number of teams from the AFC and NFC on bye each week
            - each team has exactly 1 bye week per season
    Args:
        league (list): a list of NFLTeam objects that represent all 32 teams in the NFL
    Returns:
        *** NOTE: the returned dict is just a visual aid ***
        (dict): a dictionary with key = week_num (int) and value = list of [team.name, team.conference] for each team on bye that week 
    """
    
    # Byes can only occur from week 4 to week 14
    eligible_weeks = list(range(4, 15)) 

    # keep randomly creating the number of teeams on bye for each week until there are 32 bye slots
    weekly_bye_count_list = [random.choice([2,4]) for week in eligible_weeks]
    while sum(weekly_bye_count_list) != 32:
        weekly_bye_count_list =  [random.choice([2,4]) for week in eligible_weeks]

    # add the weeks at start and end of season to fill out the full schedule
    weekly_bye_count_list = [0,0,0] + weekly_bye_count_list + [0,0,0,0]
    weekly_bye_slots_per_conference = [value//2 for value in weekly_bye_count_list]

    # print(weekly_bye_count_list, weekly_bye_slots_per_conference)
    
    # random.shuffle(league)

    # Separate the AFC and NFC teams to ensure same number of teams 
    #  from each conference have a bye each week.
    AFC_teams = [ team for team in league if "AFC" in team.conference ]
    NFC_teams = [ team for team in league if "NFC" in team.conference ]

    # shuffle the conferences to randomize when each team gets the bye week
    random.shuffle(AFC_teams)
    random.shuffle(NFC_teams)

    for week_num in range(len(weekly_bye_count_list)):
        # adjusting 0 index lists in python to fit real world terms
        real_week_num = week_num + 1

        for num_slots in range(weekly_bye_slots_per_conference[week_num]):
            # pop 1 team from the AFC and 1 from the NFC
            afcteam = AFC_teams.pop()
            nfcteam = NFC_teams.pop()
            
            afcteam.set_bye_week(real_week_num)
            nfcteam.set_bye_week(real_week_num)
            # print("\t", afcteam.name, afcteam.bye_week, "--", nfcteam.name, nfcteam.bye_week)

    # print the results for quality control purposes
    print({week: [[team, team.conference] for team in league if team.bye_week == week] for week in eligible_weeks})
    # return bye_week_summary

In [4]:
# initialize an object for all 32 NFL Teams
# ----------------------------------------------------------------------------------------------------------------------- #
# ----------------------------------------------------------------------------------------------------------------------- #
# def create_new_league():

# AFC East Teams
patriots = NFLTeam("New England Patriots", "AFC", "East")
bills = NFLTeam("Buffalo Bills", "AFC", "East")
dolphins = NFLTeam("Miami Dolphins", "AFC", "East")
jets = NFLTeam("New York Jets", "AFC", "East")

# AFC North Teams
ravens = NFLTeam("Baltimore Ravens", "AFC", "North")
steelers = NFLTeam("Pittsburgh Steelers", "AFC", "North")
browns = NFLTeam("Cleveland Browns", "AFC", "North")
bengals = NFLTeam("Cincinnati Bengals", "AFC", "North")

# AFC South Teams
texans = NFLTeam("Houston Texans", "AFC", "South")
colts = NFLTeam("Indianapolis Colts", "AFC", "South")
titans = NFLTeam("Tennessee Titans", "AFC", "South")
jaguars = NFLTeam("Jacksonville Jaguars", "AFC", "South")

# AFC West Teams
chiefs = NFLTeam("Kansas City Chiefs", "AFC", "West")
broncos = NFLTeam("Denver Broncos", "AFC", "West")
raiders = NFLTeam("Las Vegas Raiders", "AFC", "West")
chargers = NFLTeam("Los Angeles Chargers", "AFC", "West")

# NFC East Teams
cowboys = NFLTeam("Dallas Cowboys", "NFC", "East")
washington = NFLTeam("Washington Football Team", "NFC", "East")
eagles = NFLTeam("Philadelphia Eagles", "NFC", "East")
giants = NFLTeam("New York Giants", "NFC", "East")

# NFC North Teams
packers = NFLTeam("Green Bay Packers", "NFC", "North")
bears = NFLTeam("Chicago Bears", "NFC", "North")
vikings = NFLTeam("Minnesota Vikings", "NFC", "North")
lions = NFLTeam("Detroit Lions", "NFC", "North")

# NFC South Teams
buccaneers = NFLTeam("Tampa Bay Buccaneers", "NFC", "South")
saints = NFLTeam("New Orleans Saints", "NFC", "South")
panthers = NFLTeam("Carolina Panthers", "NFC", "South")
falcons = NFLTeam("Atlanta Falcons", "NFC", "South")

# NFC West Teams
seahawks = NFLTeam("Seattle Seahawks", "NFC", "West")
rams = NFLTeam("Los Angeles Rams", "NFC", "West")
cardinals = NFLTeam("Arizona Cardinals", "NFC", "West")
sf49ers = NFLTeam("San Francisco 49ers", "NFC", "West")

# Create a list of all 32 NFLTeam objects
nfl_teams = [
    patriots, bills, dolphins, jets,
    ravens, steelers, browns, bengals,
    texans, colts, titans, jaguars,
    chiefs, broncos, raiders, chargers,
    cowboys, washington, eagles, giants,
    packers, bears, vikings, lions,
    buccaneers, saints, panthers, falcons,
    seahawks, rams, cardinals, sf49ers
]

# return nfl_teams

# Create a list of teams in the AFC and NFC
AFC = [ team for team in nfl_teams if "AFC" in team.conference ]
NFC = [ team for team in nfl_teams if "NFC" in team.conference ]
# [team.name for team in AFC]

In [5]:
# ASSIGNING BYE WEEKS

print("Assigning bye weeks: \n")
assign_bye_weeks(nfl_teams)

Assigning bye weeks: 

{4: [[Indianapolis Colts AFC.s, 'AFC'], [Tampa Bay Buccaneers NFC.s, 'NFC']], 5: [[Jacksonville Jaguars AFC.s, 'AFC'], [Los Angeles Chargers AFC.w, 'AFC'], [New York Giants NFC.e, 'NFC'], [Minnesota Vikings NFC.n, 'NFC']], 6: [[Buffalo Bills AFC.e, 'AFC'], [Pittsburgh Steelers AFC.n, 'AFC'], [Detroit Lions NFC.n, 'NFC'], [San Francisco 49ers NFC.w, 'NFC']], 7: [[Miami Dolphins AFC.e, 'AFC'], [New York Jets AFC.e, 'AFC'], [Dallas Cowboys NFC.e, 'NFC'], [Los Angeles Rams NFC.w, 'NFC']], 8: [[Cleveland Browns AFC.n, 'AFC'], [Washington Football Team NFC.e, 'NFC']], 9: [[New England Patriots AFC.e, 'AFC'], [Las Vegas Raiders AFC.w, 'AFC'], [Chicago Bears NFC.n, 'NFC'], [Atlanta Falcons NFC.s, 'NFC']], 10: [[Kansas City Chiefs AFC.w, 'AFC'], [Arizona Cardinals NFC.w, 'NFC']], 11: [[Baltimore Ravens AFC.n, 'AFC'], [Houston Texans AFC.s, 'AFC'], [New Orleans Saints NFC.s, 'NFC'], [Seattle Seahawks NFC.w, 'NFC']], 12: [[Tennessee Titans AFC.s, 'AFC'], [Green Bay Packers 

In [6]:
nfl_teams

[New England Patriots AFC.e,
 Buffalo Bills AFC.e,
 Miami Dolphins AFC.e,
 New York Jets AFC.e,
 Baltimore Ravens AFC.n,
 Pittsburgh Steelers AFC.n,
 Cleveland Browns AFC.n,
 Cincinnati Bengals AFC.n,
 Houston Texans AFC.s,
 Indianapolis Colts AFC.s,
 Tennessee Titans AFC.s,
 Jacksonville Jaguars AFC.s,
 Kansas City Chiefs AFC.w,
 Denver Broncos AFC.w,
 Las Vegas Raiders AFC.w,
 Los Angeles Chargers AFC.w,
 Dallas Cowboys NFC.e,
 Washington Football Team NFC.e,
 Philadelphia Eagles NFC.e,
 New York Giants NFC.e,
 Green Bay Packers NFC.n,
 Chicago Bears NFC.n,
 Minnesota Vikings NFC.n,
 Detroit Lions NFC.n,
 Tampa Bay Buccaneers NFC.s,
 New Orleans Saints NFC.s,
 Carolina Panthers NFC.s,
 Atlanta Falcons NFC.s,
 Seattle Seahawks NFC.w,
 Los Angeles Rams NFC.w,
 Arizona Cardinals NFC.w,
 San Francisco 49ers NFC.w]

In [7]:
for team in nfl_teams:
    team.opponents_not_faced = [team for team in nfl_teams]
    # team.opponents_not_faced.remove(team.name)

bengals.opponents_not_faced

[New England Patriots AFC.e,
 Buffalo Bills AFC.e,
 Miami Dolphins AFC.e,
 New York Jets AFC.e,
 Baltimore Ravens AFC.n,
 Pittsburgh Steelers AFC.n,
 Cleveland Browns AFC.n,
 Cincinnati Bengals AFC.n,
 Houston Texans AFC.s,
 Indianapolis Colts AFC.s,
 Tennessee Titans AFC.s,
 Jacksonville Jaguars AFC.s,
 Kansas City Chiefs AFC.w,
 Denver Broncos AFC.w,
 Las Vegas Raiders AFC.w,
 Los Angeles Chargers AFC.w,
 Dallas Cowboys NFC.e,
 Washington Football Team NFC.e,
 Philadelphia Eagles NFC.e,
 New York Giants NFC.e,
 Green Bay Packers NFC.n,
 Chicago Bears NFC.n,
 Minnesota Vikings NFC.n,
 Detroit Lions NFC.n,
 Tampa Bay Buccaneers NFC.s,
 New Orleans Saints NFC.s,
 Carolina Panthers NFC.s,
 Atlanta Falcons NFC.s,
 Seattle Seahawks NFC.w,
 Los Angeles Rams NFC.w,
 Arizona Cardinals NFC.w,
 San Francisco 49ers NFC.w]

In [75]:
# bengals.add_game(2,chargers)
# bengals.schedule

In [76]:
myschedule = NFLSchedule(weeks=18)
full_schedule = {}


for week_num in range(0,19):
# for week_num in range(4,5):
    weekly_matchups = []
    weekly_matchups_names = []

    # list of teams in afc and nfc that are playing a game this week
    afc_eligible_teams = [team for team in AFC if team.bye_week != week_num]
    nfc_eligible_teams = [team for team in NFC if team.bye_week != week_num]

    # shuffle the teams to get randomized matchups
    random.shuffle(afc_eligible_teams)
    random.shuffle(nfc_eligible_teams)

    print(f"\n{week_num = }")
    # print(f"{afc_eligible_teams = }")
    # print(f"{nfc_eligible_teams = }")

    for team in nfl_teams:
        opponent = random.choice(team.opponents_not_faced)
        # team.opponents_not_faced.remove(opponent)
        

    # as if every team plays someone from the other conference
    for afc_team, nfc_team in zip(afc_eligible_teams, nfc_eligible_teams):
        weekly_matchups.append({"home": afc_team, "away": nfc_team})
        weekly_matchups_names.append({"home": afc_team.name, "away": nfc_team.name})
    # print(f"{weekly_matchups_names = }")


    # assign that game to each team's 
    for game in weekly_matchups:
        # print(game)
        home_team = game["home"]
        away_team = game["away"]

        print(away_team, "@", home_team)

        home_team.add_game(week_num, away_team)
        away_team.add_game(week_num, home_team)

        # game["home"].add_game(week_num, game["away"])
        # game["away"].add_game(week_num, game["home"])
        
        # add_game_to_team(week_num, game["home"])
        # add_game_to_team(week_num, game["away"])
        
    full_schedule[f"week_{week_num+1}"] = weekly_matchups
    
# full_schedule_json = json.dumps(full_schedule, indent=4)  # indent for pretty printing
# print(full_schedule_json)
# full_schedule



week_num = 0
Carolina Panthers NFC.s @ Cleveland Browns AFC.n
Green Bay Packers NFC.n @ Las Vegas Raiders AFC.w
Dallas Cowboys NFC.e @ Cincinnati Bengals AFC.n
New York Giants NFC.e @ Pittsburgh Steelers AFC.n
Chicago Bears NFC.n @ New York Jets AFC.e
Tampa Bay Buccaneers NFC.s @ New England Patriots AFC.e
San Francisco 49ers NFC.w @ Los Angeles Chargers AFC.w
New Orleans Saints NFC.s @ Baltimore Ravens AFC.n
Detroit Lions NFC.n @ Indianapolis Colts AFC.s
Los Angeles Rams NFC.w @ Jacksonville Jaguars AFC.s
Atlanta Falcons NFC.s @ Denver Broncos AFC.w
Washington Football Team NFC.e @ Miami Dolphins AFC.e
Philadelphia Eagles NFC.e @ Houston Texans AFC.s
Minnesota Vikings NFC.n @ Kansas City Chiefs AFC.w
Seattle Seahawks NFC.w @ Tennessee Titans AFC.s
Arizona Cardinals NFC.w @ Buffalo Bills AFC.e

week_num = 1
New Orleans Saints NFC.s @ Tennessee Titans AFC.s
Minnesota Vikings NFC.n @ Houston Texans AFC.s
New York Giants NFC.e @ New York Jets AFC.e
Atlanta Falcons NFC.s @ Buffalo Bills A

In [77]:
# [game.name if type(game) == NFLTeam else game for game in bengals.schedule.values()]
# [game.name if type(game) == NFLTeam else game for game in sf49ers.schedule.values()]

[game.name if type(game) == NFLTeam else game for game in patriots.schedule.values()]
# [game.name if type(game) == NFLTeam else game for game in cowboys.schedule.values()]

['Tampa Bay Buccaneers',
 'Washington Football Team',
 'Washington Football Team',
 'Green Bay Packers',
 ' -- BYE WEEK --',
 'Seattle Seahawks',
 'Philadelphia Eagles',
 'Chicago Bears',
 'Arizona Cardinals',
 'New York Giants',
 'Minnesota Vikings',
 'Minnesota Vikings',
 'Los Angeles Rams',
 'Detroit Lions',
 'Green Bay Packers',
 'Washington Football Team',
 'Philadelphia Eagles',
 'Philadelphia Eagles',
 'New York Giants']